In [26]:
import pandas as pd 
import pm4py
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

In [7]:
#log = pm4py.read_xes("../data/InternationalDeclarations.xes.gz")
#df = pm4py.convert_to_dataframe(log)

## Load Data Preprocessed in ProM

In [8]:
log_time = pm4py.read_xes("../data/International_Declarations_final.xes")
df_time = pm4py.convert_to_dataframe(log_time)
#log_time = pm4py.read_xes('../data/International_Declarations_final.xes')

parsing log, completed traces :: 100%|██████████| 5646/5646 [00:04<00:00, 1253.48it/s]


In [9]:
working_df = df_time
print(len(working_df.index))
working_df.head()

65376


,org:resource,org:role,remaining_time,id,concept:name,time:timestamp,case:Permit travel permit number,case:DeclarationNumber,case:Amount,case:RequestedAmount,...,case:id,case:Permit ID,case:Permit id,case:BudgetNumber,case:Permit ActivityNumber,case:AdjustedAmount,case:concept:name,trip_duration,remaining_time_since_end_trip,time_submission
0,STAFF MEMBER,EMPLOYEE,4577.514722,rv_travel permit 76455_6,Start trip,2016-10-05 00:00:00+02:00,travel permit number 76456,declaration number 76458,39.664561,39.664561,...,declaration 76457,travel permit 76455,travel permit 76455,budget 144133,activity 46005,39.664561,declaration 76457,NaN,NaN,NaN
1,STAFF MEMBER,EMPLOYEE,4577.514722,rv_travel permit 76455_7,End trip,2016-10-05 00:00:00+02:00,travel permit number 76456,declaration number 76458,39.664561,39.664561,...,declaration 76457,travel permit 76455,travel permit 76455,budget 144133,activity 46005,39.664561,declaration 76457,0.0,4577.514722,NaN
2,STAFF MEMBER,EMPLOYEE,171.978611,st_step 76459_0,Permit SUBMITTED by EMPLOYEE,2017-04-06 13:32:10+02:00,travel permit number 76456,declaration number 76458,39.664561,39.664561,...,declaration 76457,travel permit 76455,travel permit 76455,budget 144133,activity 46005,39.664561,declaration 76457,NaN,NaN,NaN
3,STAFF MEMBER,SUPERVISOR,171.973611,st_step 76460_0,Permit FINAL_APPROVED by SUPERVISOR,2017-04-06 13:32:28+02:00,travel permit number 76456,declaration number 76458,39.664561,39.664561,...,declaration 76457,travel permit 76455,travel permit 76455,budget 144133,activity 46005,39.664561,declaration 76457,NaN,NaN,NaN
4,STAFF MEMBER,EMPLOYEE,147.877500,st_step 76461_0,Declaration SUBMITTED by EMPLOYEE,2017-04-07 13:38:14+02:00,travel permit number 76456,declaration number 76458,39.664561,39.664561,...,declaration 76457,travel permit 76455,travel permit 76455,budget 144133,activity 46005,39.664561,declaration 76457,NaN,NaN,NaN


## Remove all data recorded after "End trip"
* We do not do prefix abstraction in this case, as we are specifically interested in predicting the remaining time after a specific event (End Trip) has happend
* Both for training and test data we thus remove this information
* The posterior feature (remaining time) was already computed in ProM for the "End trip" event

In [10]:
eventColumnName = "concept:name"
caseIdColumnName = "case:id"
caseIds = working_df[caseIdColumnName].unique()

index_before_End = []

for caseId in caseIds:
    cur_trace = working_df[working_df[caseIdColumnName] == caseId]
    end_trip_time = pd.Timestamp(cur_trace[cur_trace[eventColumnName] == "End trip"]["time:timestamp"].values[0])
    
    for i, event in cur_trace.iterrows():
        if pd.Timestamp(event["time:timestamp"]).replace(tzinfo=None) <= end_trip_time.tz_localize(None):
            index_before_End.append(i)
        
df_event_before_end = working_df.iloc[index_before_End].copy()
df_event_before_end.head()

,org:resource,org:role,remaining_time,id,concept:name,time:timestamp,case:Permit travel permit number,case:DeclarationNumber,case:Amount,case:RequestedAmount,...,case:id,case:Permit ID,case:Permit id,case:BudgetNumber,case:Permit ActivityNumber,case:AdjustedAmount,case:concept:name,trip_duration,remaining_time_since_end_trip,time_submission
0,STAFF MEMBER,EMPLOYEE,4577.514722,rv_travel permit 76455_6,Start trip,2016-10-05 00:00:00+02:00,travel permit number 76456,declaration number 76458,39.664561,39.664561,...,declaration 76457,travel permit 76455,travel permit 76455,budget 144133,activity 46005,39.664561,declaration 76457,NaN,NaN,NaN
1,STAFF MEMBER,EMPLOYEE,4577.514722,rv_travel permit 76455_7,End trip,2016-10-05 00:00:00+02:00,travel permit number 76456,declaration number 76458,39.664561,39.664561,...,declaration 76457,travel permit 76455,travel permit 76455,budget 144133,activity 46005,39.664561,declaration 76457,0.0,4577.514722,NaN
8,STAFF MEMBER,EMPLOYEE,2705.517222,rv_travel permit 76665_6,Start trip,2016-11-21 00:00:00+01:00,travel permit number 76666,declaration number 76668,346.544903,346.544903,...,declaration 76667,travel permit 76665,travel permit 76665,budget 144054,UNKNOWN,346.544903,declaration 76667,NaN,NaN,NaN
9,STAFF MEMBER,EMPLOYEE,1961.517222,rv_travel permit 76665_7,End trip,2016-12-22 00:00:00+01:00,travel permit number 76666,declaration number 76668,346.544903,346.544903,...,declaration 76667,travel permit 76665,travel permit 76665,budget 144054,UNKNOWN,346.544903,declaration 76667,744.0,1961.517222,NaN
16,STAFF MEMBER,EMPLOYEE,1025.530556,rv_travel permit 73652_6,Start trip,2016-12-08 00:00:00+01:00,travel permit number 73653,declaration number 73655,56.972769,56.972769,...,declaration 73654,travel permit 73652,travel permit 73652,budget 143677,UNKNOWN,56.972769,declaration 73654,NaN,NaN,NaN


## Naive Frequency Based Encoding
Frequency based encoding on all events

ToDo: Encoding only on Events before End Trip

In [11]:
eventCategories = df_event_before_end[eventColumnName].unique()
caseIds = df_event_before_end[caseIdColumnName].unique()

frequency_df = pd.DataFrame(index=caseIds, columns=eventCategories).fillna(0)

for cur_case in caseIds:
    cur_eventCategories = df_event_before_end[df_event_before_end[caseIdColumnName] == cur_case][eventColumnName]
    for e in cur_eventCategories:
        frequency_df.loc[cur_case][e] += 1  

In [12]:
frequency_df.head()

,Start trip,End trip,Permit SUBMITTED by EMPLOYEE,Permit APPROVED by SUPERVISOR,Permit FINAL_APPROVED by DIRECTOR,Permit FINAL_APPROVED by SUPERVISOR,Permit APPROVED by PRE_APPROVER,Permit REJECTED by SUPERVISOR,Permit REJECTED by EMPLOYEE,Permit REJECTED by MISSING,...,Declaration REJECTED by SUPERVISOR,Permit APPROVED by ADMINISTRATION,Permit APPROVED by BUDGET OWNER,Declaration APPROVED by ADMINISTRATION,Declaration APPROVED by BUDGET OWNER,Declaration REJECTED by ADMINISTRATION,Declaration REJECTED by EMPLOYEE,Permit REJECTED by ADMINISTRATION,Permit REJECTED by BUDGET OWNER,Declaration REJECTED by BUDGET OWNER
declaration 76457,1,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
declaration 76667,1,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
declaration 73654,1,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
declaration 73582,1,1,1,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
declaration 72590,1,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## Add some interresting features to the dataset

We first compute these features in the initial prefix dataset 

In [13]:
df_event_before_end["time_submission"] = df_event_before_end.groupby("case:concept:name")["time_submission"].transform(lambda x: x.fillna(x.mean()))
df_event_before_end["time_submission"] = df_event_before_end.groupby("case:concept:name")["time_submission"].transform(lambda x: x.fillna(0))
df_event_before_end["trip_duration"] = df_event_before_end.groupby("case:concept:name")["trip_duration"].transform(lambda x: x.fillna(x.mean()))
df_event_before_end["remaining_time_since_end_trip"] = df_event_before_end.groupby("case:concept:name")["remaining_time_since_end_trip"].transform(lambda x: x.fillna(x.mean()))

Then we add these new features to the frequency based encoder dataset

In [14]:
time_sub = []
trip_duration = []
remaining_time = []
start_time_event = []
amount = []


for caseId in caseIds:
    cur_trace = df_event_before_end.loc[df_event_before_end[caseIdColumnName] == caseId]
    time_sub.append(cur_trace["time_submission"].iloc[0])
    trip_duration.append(cur_trace["trip_duration"].iloc[0])
    remaining_time.append(cur_trace["remaining_time_since_end_trip"].iloc[0])
    start_time_event.append(pd.Timestamp(cur_trace["time:timestamp"].iloc[0]))
    amount.append(cur_trace["case:Amount"].iloc[0])

In [15]:
frequency_df["start_time_trace"] = start_time_event
frequency_df["remaining_time_since_end_trip"] = remaining_time
frequency_df["trip_duration"] = trip_duration
frequency_df["time_submission"] = time_sub
frequency_df["amount"] = amount

We sort the dataset by the start time of each traces (useful to split the data into a training and a test set)

In [16]:
frequency_df.sort_values(by='start_time_trace')

,Start trip,End trip,Permit SUBMITTED by EMPLOYEE,Permit APPROVED by SUPERVISOR,Permit FINAL_APPROVED by DIRECTOR,Permit FINAL_APPROVED by SUPERVISOR,Permit APPROVED by PRE_APPROVER,Permit REJECTED by SUPERVISOR,Permit REJECTED by EMPLOYEE,Permit REJECTED by MISSING,...,Declaration REJECTED by ADMINISTRATION,Declaration REJECTED by EMPLOYEE,Permit REJECTED by ADMINISTRATION,Permit REJECTED by BUDGET OWNER,Declaration REJECTED by BUDGET OWNER,start_time_trace,remaining_time_since_end_trip,trip_duration,time_submission,amount
declaration 76457,1,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,2016-10-05 00:00:00+02:00,4577.514722,0.0,0.000000,39.664561
declaration 76667,1,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,2016-11-21 00:00:00+01:00,1961.517222,744.0,0.000000,346.544903
declaration 73654,1,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,2016-12-08 00:00:00+01:00,1025.530556,0.0,0.000000,56.972769
declaration 73582,1,1,1,1,1,0,0,0,0,0,...,0,0,0,0,0,2017-01-01 00:00:00+01:00,617.518889,8736.0,0.000000,199.546260
declaration 72590,1,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,2017-01-09 00:00:00+01:00,809.538333,48.0,0.000000,14.569044
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
declaration 12674,1,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,2018-12-07 13:00:16+01:00,1361.522500,24.0,34.995556,43.782713
declaration 15973,1,1,1,0,0,1,0,0,0,0,...,0,0,0,0,0,2018-12-07 22:32:45+01:00,1001.528889,48.0,97.454167,925.091226
declaration 12689,1,1,1,0,0,1,0,0,0,0,...,0,0,0,0,0,2018-12-09 17:41:05+01:00,419.987778,24.0,150.315278,297.421020
declaration 15436,1,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,2018-12-10 00:00:00+01:00,809.522222,48.0,0.000000,317.596586


## Split the dataset into a training set and a test set

We chose to not consider the overlapping of the different traces and we take the first 80% of the traces according to their starting date.

In [22]:
n = len(start_time_event)
train_set_size = int(n*0.8)

x_train = frequency_df.iloc[:train_set_size, :].drop(["remaining_time_since_end_trip","start_time_trace"], axis=1)
y_train = remaining_time[:train_set_size]
x_test = frequency_df.iloc[train_set_size:, :].drop(["remaining_time_since_end_trip","start_time_trace"], axis=1)
y_test = remaining_time[train_set_size:]

## Training of the model

Now that the data are cleaned and preprocessed, we can start to train our model.

In [37]:
from sklearn.ensemble import RandomForestRegressor

rf = RandomForestRegressor(n_estimators=1000,random_state=42)
rf.fit(x_train, y_train)

RandomForestRegressor(n_estimators=1000, random_state=42)

**Evaluate the model**

In [38]:
from sklearn.metrics import mean_squared_error, mean_absolute_error
#Use the random forest model to predict the outcome (remaining time since the trip ended)
predictions = rf.predict(x_test)


#Print the mean absolute error (MAE) and Mean Square Error (MSE)
print('Mean Absolute Error: ', round(mean_absolute_error(y_test, predictions), 2), 'hours.')

print('Mean Square Error: ', round(mean_squared_error(y_test, predictions), 2), ' hours.')

Mean Absolute Error:  425.13 hours.
Mean Square Error:  372861.78  hours.


Try to reduce the number of useless variables and just take the most important one by measuring their importance

In [40]:
feature_list = x_train.columns
# Get numerical feature importances
importances = list(rf.feature_importances_)# List of tuples with variable and importance
feature_importances = [(feature, round(importance, 2)) for feature, importance in zip(feature_list, importances)]# Sort the feature importances by most important first
feature_importances = sorted(feature_importances, key = lambda x: x[1], reverse = True)# Print out the feature and importances 
[print('Variable: {:20} Importance: {}'.format(*pair)) for pair in feature_importances]

Variable: amount               Importance: 0.45
Variable: time_submission      Importance: 0.31
Variable: trip_duration        Importance: 0.14
Variable: Permit APPROVED by ADMINISTRATION Importance: 0.03
Variable: Permit SUBMITTED by EMPLOYEE Importance: 0.02
Variable: Permit FINAL_APPROVED by SUPERVISOR Importance: 0.01
Variable: Permit APPROVED by PRE_APPROVER Importance: 0.01
Variable: Declaration SUBMITTED by EMPLOYEE Importance: 0.01
Variable: Permit APPROVED by BUDGET OWNER Importance: 0.01
Variable: Start trip           Importance: 0.0
Variable: End trip             Importance: 0.0
Variable: Permit APPROVED by SUPERVISOR Importance: 0.0
Variable: Permit FINAL_APPROVED by DIRECTOR Importance: 0.0
Variable: Permit REJECTED by SUPERVISOR Importance: 0.0
Variable: Permit REJECTED by EMPLOYEE Importance: 0.0
Variable: Permit REJECTED by MISSING Importance: 0.0
Variable: Declaration APPROVED by PRE_APPROVER Importance: 0.0
Variable: Declaration FINAL_APPROVED by SUPERVISOR Importance

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]

In [48]:
rf_most_important = RandomForestRegressor(n_estimators=1000, random_state=42)

important_indices = ['amount', 'time_submission', 'trip_duration']
train_important = x_train.loc[:, important_indices]
test_important = x_test.loc[:, important_indices]

rf_most_important.fit(train_important, y_train)

predictions2 = rf_most_important.predict(test_important)

print('Mean Absolute Error: ', round(mean_absolute_error(y_test, predictions2), 2), 'hours.')

print('Mean Square Error: ', round(mean_squared_error(y_test, predictions2), 2), ' hours.')



Mean Absolute Error:  451.19 hours.
Mean Square Error:  425480.02  hours.


**Try to find the best hyperparameters with randomized grid search**

In [49]:
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

In [50]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV


rf_hp = RandomForestRegressor()
rf_random = RandomizedSearchCV(estimator=rf_hp, param_distributions=random_grid, n_iter=100, cv=3, verbose=2, random_state=42, n_jobs=-1)
rf_random.fit(x_train, y_train)

Fitting 3 folds for each of 100 candidates, totalling 300 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:  1.6min
[Parallel(n_jobs=-1)]: Done 146 tasks      | elapsed:  6.8min
[Parallel(n_jobs=-1)]: Done 300 out of 300 | elapsed: 15.4min finished


RandomizedSearchCV(cv=3, estimator=RandomForestRegressor(), n_iter=100,
                   n_jobs=-1,
                   param_distributions={'bootstrap': [True, False],
                                        'max_depth': [10, 20, 30, 40, 50, 60,
                                                      70, 80, 90, 100, 110,
                                                      None],
                                        'max_features': ['auto', 'sqrt'],
                                        'min_samples_leaf': [1, 2, 4],
                                        'min_samples_split': [2, 5, 10],
                                        'n_estimators': [200, 400, 600, 800,
                                                         1000, 1200, 1400, 1600,
                                                         1800, 2000]},
                   random_state=42, verbose=2)

In [51]:
rf_random.best_params_

{'n_estimators': 400,
 'min_samples_split': 2,
 'min_samples_leaf': 4,
 'max_features': 'sqrt',
 'max_depth': 10,
 'bootstrap': True}

In [53]:
len(feature_list)

30

Grid Search with cross validation

In [57]:
param_grid = {
    'bootstrap': [True],
    'max_depth': [10,15,20],
    'max_features': [5,6],
    'min_samples_leaf': [3, 4, 5],
    'min_samples_split': [1,2,3],
    'n_estimators': [100, 200, 300, 1000]
}

rf_opt = RandomForestRegressor()

grid_search = GridSearchCV(estimator=rf, param_grid= param_grid, cv=3, n_jobs=-1, verbose=2)

grid_search.fit(x_train, y_train)

Fitting 3 folds for each of 216 candidates, totalling 648 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:   13.9s
[Parallel(n_jobs=-1)]: Done 146 tasks      | elapsed:   47.7s
[Parallel(n_jobs=-1)]: Done 349 tasks      | elapsed:  1.8min
[Parallel(n_jobs=-1)]: Done 632 tasks      | elapsed:  3.2min
[Parallel(n_jobs=-1)]: Done 648 out of 648 | elapsed:  3.3min finished


GridSearchCV(cv=3,
             estimator=RandomForestRegressor(n_estimators=1000,
                                             random_state=42),
             n_jobs=-1,
             param_grid={'bootstrap': [True], 'max_depth': [10, 15, 20],
                         'max_features': [5, 6], 'min_samples_leaf': [3, 4, 5],
                         'min_samples_split': [1, 2, 3],
                         'n_estimators': [100, 200, 300, 1000]},
             verbose=2)

In [59]:
best_grid = grid_search.best_estimator_

predictions_opt = grid_search.predict(x_test)

print('Mean Absolute Error: ', round(mean_absolute_error(y_test, predictions_opt), 2), 'hours.')

print('Mean Square Error: ', round(mean_squared_error(y_test, predictions_opt), 2), ' hours.')

Mean Absolute Error:  387.49 hours.
Mean Square Error:  264909.03  hours.
